In [10]:
#import librairies

from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize


In [48]:
#Here is the URL we are going to scrape: we are going to scrape the Liberation media page: 
#we are looking for articles about "science" or "l'intelligence artificielle"
topic = input('enter your search')
topic = topic.replace(" ", "+").replace("'", "%27")
url = 'https://www.liberation.fr/recherche/?q=' + topic
response = requests.get(url).content
response

enter your searchintelligence artificielle


b'<!DOCTYPE html>\n\n\n\n\n\n\n\n\n\n\n\n<html lang="fr">\n  <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">\n    <title>intelligence artificielle - Recherche - Lib\xc3\xa9ration</title>\n\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="description" content="Tout savoir sur les derni\xc3\xa8res actualit\xc3\xa9s politiques, monde, soci\xc3\xa9t\xc3\xa9, sports, \xc3\xa9cologie avec le journal en ligne Lib\xc3\xa9ration.">\n    <meta name="keywords" content="actualit\xc3\xa9s, news">\n    <meta name="apple-itunes-app" content="app-id=371288979">\n\n    \n\n    \n      <meta name="google-site-verification" content="F49Mb6L-Rs_uQg4KRSIoZq4tN4d6vltGt_QqCVFfjYw">\n      <meta name="google-site-verification" content="ZU8yhvOSn18Hu1fXjykeuXzDBFFdwNN6ah6Km9zr85k">\n    \n\n    \n      <meta property="fb:pages" content="147126052393" />\n    \n\n    <meta property="og:site_na

In [49]:
#We use BeautifulSoup library to read the raw HTML and structure it in a way where we will be able to more easily parse the information we want out of it
from bs4 import BeautifulSoup
soup = BeautifulSoup(response,"lxml")
print(soup)


<!DOCTYPE html>
<html lang="fr">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<title>intelligence artificielle - Recherche - Libération</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Tout savoir sur les dernières actualités politiques, monde, société, sports, écologie avec le journal en ligne Libération." name="description"/>
<meta content="actualités, news" name="keywords"/>
<meta content="app-id=371288979" name="apple-itunes-app"/>
<meta content="F49Mb6L-Rs_uQg4KRSIoZq4tN4d6vltGt_QqCVFfjYw" name="google-site-verification"/>
<meta content="ZU8yhvOSn18Hu1fXjykeuXzDBFFdwNN6ah6Km9zr85k" name="google-site-verification"/>
<meta content="147126052393" property="fb:pages"/>
<meta content="Libération.fr" property="og:site_name"/>
<meta content="website" property="og:type"/>
<meta content="" property="og:title"/>
<meta content="https://statics.liberation.fr/newsite/images/

In [50]:
#We create a list of the titles corresponding to titles mentioning Science, we can see on the website that we have 753 c
web_content_list = []

# To extract the title and the resume :
title_resume = soup.find_all("div",{"class" :"live-content"})

# To extract the date of each publications
dates = soup.find_all("div", {"class":"live-aside"})
    
# To process article by article by looping:
for title, date in zip(title_resume, dates):
    
# To store the information to a dictionary
    web_content_dict = {}
    web_content_dict['Title'] = title.find('h3',{'class': 'live-title'}).text.replace("\r","").replace("\n","")
    web_content_dict['Resume'] = title.find("p").text
    web_content_dict['Date'] = date.find('a').text
    
      
      # To store the dictionary to into a list
    web_content_list.append(web_content_dict)

web_content_list
                  
                  
                  

[{'Title': 'Vibro, c’est trop',
  'Resume': 'Banalisé, décliné pour\xa0tous les goûts… le\xa0gode en a fait, du\xa0chemin, depuis l’époque victorienne. Faut-il pour autant qu’il\xa0compte nos\xa0calories et\xa0se\xa0connecte au\xa0Bluetooth ?',
  'Date': '17.08.19'},
 {'Title': 'Oups, Facebook admet avoir écouté des conversations privées',
  'Resume': "Le réseau social de Mark Zuckerberg a longtemps nié écouter les conversations de ses utilisateurs. Selon Bloomberg, c'est pourtant le cas. L'entreprise a recours à des sous-traitants pour transcrire des conversations via l'application Messenger.",
  'Date': '15.08.19'},
 {'Title': 'L’ordinateur, machine à trier les hommes',
  'Resume': '«Fake\xa0news», contrôle\xa0des données, cyberguerre… Retour sur\xa0la face sombre de\xa0nos vies numériques. Aujourd’hui, comment les algorithmes peuvent devenir des outils de\xa0discrimination.',
  'Date': '01.08.19'},
 {'Title': 'Détecter le mensonge, une science hasardeuse',
  'Resume': 'Formation en 

In [51]:
#Create a dataframe to store our extracted data
df = pd.DataFrame(web_content_list)
df.columns

Index(['Date', 'Resume', 'Title'], dtype='object')

In [52]:
df = df[['Date', 'Title', 'Resume']]
df

,Date,Title,Resume
0,17.08.19,"Vibro, c’est trop","Banalisé, décliné pour tous les goûts… le gode..."
1,15.08.19,"Oups, Facebook admet avoir écouté des conversa...",Le réseau social de Mark Zuckerberg a longtemp...
2,01.08.19,"L’ordinateur, machine à trier les hommes","«Fake news», contrôle des données, cyberguerre..."
3,30.07.19,"Détecter le mensonge, une science hasardeuse","Formation en négociation, analyse des comporte..."
4,27.07.19,"Surveillance en Chine, le pire du milieu","Fake news, contrôle des données, cyberguerre… ..."
5,25.07.19,"Loi bioéthique : Allô mamans, bientôt ?",Le projet de loi qui arrivera en septembre à l...
6,25.07.19,Intelligence artificielle : IA quelqu’un ?,"Elle est partout : téléphones, assistants pers..."
7,24.07.19,"PMA pour toutes : deux mois avant les débats, ...",Alors que le projet de loi est présenté ce mer...
8,24.07.19,"Mileva Einstein, l’inconnue de l’équation",Star de l'ombre. Sans cette mathématicienne ho...
9,23.07.19,Littérature : quand la science faisait vibrer ...,"Jusqu'au 25 août, la BNF propose de découvrir ..."


In [42]:
# To save our dataframe in a csv file
Libe_publications_word= df.to_csv('libeword.csv', index=False)